In [1]:
import nltk
import scipy
import pandas as pd
import numpy as np

import csv
from nltk.corpus import wordnet as wn
from libs.sym import *

In [3]:
if __name__ == "__main__":

    #set-up of dataframe from csv file
    word_sym=pd.read_csv('data\\WordSim353.csv')
    word_sym.insert(3,"wu_palmer",np.zeros(353))
    word_sym.insert(4,"shortest_path",np.zeros(353))
    word_sym.insert(5,"leak_chod",np.zeros(353))

    for i in range(0,353):
        w1 = word_sym.at[i,'Word 1']
        w2 = word_sym.at[i,'Word 2']
        max_wu=0
        max_sp=0
        max_lch=0
        for s1 in wn.synsets(w1):
            for s2 in wn.synsets(w2):
                #similarity for specific senses of w1 and w2
                actual_wu=wu_similarity(s1,s2)
                actual_sp=sp_similarity(s1,s2)
                actual_lch=lch_similarity(s1,s2)
                #senses of w1 and w2 that maximize the similarity value
                if actual_wu>max_wu:
                    max_wu=actual_wu
                if actual_sp>max_sp:
                    max_sp=actual_sp
                if actual_lch>max_lch:
                    max_lch=actual_lch
        word_sym.iat[i,3]= max_wu
        word_sym.iat[i,4]= max_sp
        word_sym.iat[i,5]= max_lch

    #min-max normalization of similarity measures
    word_sym['wu_palmer']=(word_sym['wu_palmer']-word_sym['wu_palmer'].min())/(word_sym['wu_palmer'].max()-word_sym['wu_palmer'].min())*10
    word_sym['shortest_path']=(word_sym['shortest_path']-word_sym['shortest_path'].min())/(word_sym['shortest_path'].max()-word_sym['shortest_path'].min())*10
    word_sym['leak_chod']=(word_sym['leak_chod']-word_sym['leak_chod'].min())/(word_sym['leak_chod'].max()-word_sym['leak_chod'].min())*10
    
    print(word_sym)

    #pearson and spearman corellation of similarity measures and human values
    results=pd.DataFrame({'similarity':['Wu & Palmer','Shortest Path','Leak & Chod'],
                         'pearson':np.zeros(3),
                         'spearman':np.zeros(3)})
    results.iat[0,1]=scipy.stats.pearsonr(word_sym['Human (mean)'],word_sym['wu_palmer'],alternative='two-sided', method=None)[0]
    results.iat[1,1]=scipy.stats.pearsonr(word_sym['Human (mean)'],word_sym['shortest_path'],alternative='two-sided', method=None)[0]
    results.iat[2,1]=scipy.stats.pearsonr(word_sym['Human (mean)'],word_sym['leak_chod'],alternative='two-sided', method=None)[0]
    results.iat[0,2]=scipy.stats.spearmanr(word_sym['Human (mean)'],word_sym['wu_palmer'], axis=0, nan_policy='propagate', alternative='two-sided')[0]
    results.iat[1,2]=scipy.stats.spearmanr(word_sym['Human (mean)'],word_sym['shortest_path'], axis=0, nan_policy='propagate', alternative='two-sided')[0]
    results.iat[2,2]=scipy.stats.spearmanr(word_sym['Human (mean)'],word_sym['leak_chod'], axis=0, nan_policy='propagate', alternative='two-sided')[0]

    display(results)





           Word 1    Word 2  Human (mean)  wu_palmer  shortest_path  leak_chod
0            love       sex          6.77   8.460128       9.736842   9.929098
1           tiger       cat          7.35   8.854262       9.736842   9.929098
2           tiger     tiger         10.00   9.165903      10.000000  10.000000
3            book     paper          7.46   8.020165       9.473684   8.037094
4        computer  keyboard          7.62   7.552704       9.210526   6.930343
..            ...       ...           ...        ...            ...        ...
348        shower     flood          6.03   5.829514       8.947368   6.145091
349       weather  forecast          8.34   1.219065       6.578947   2.927852
350      disaster      area          6.25   4.582951       7.894737   4.253087
351      governor    office          6.34   4.821265       7.631579   3.931588
352  architecture   century          3.78   2.823098       7.631579   3.931588

[353 rows x 6 columns]


,similarity,pearson,spearman
0,Wu & Palmer,0.244442,0.298227
1,Shortest Path,0.061495,0.261141
2,Leak & Chod,0.272024,0.261141
